## Project Objective

In this project, I examine the single rider line at Tiana's Bayou Adventure (fomerly known as Splash Mountain) at Disneyland in Anaheim, California. I create a model to estimate the waiting time in the single rider line for any given queue length (number of guests in the queue).


## Queues Types at Disneyland

Attractions at Disneyland frequently have multiple queues that guests may enter to wait for their turn to ride an attraction. These include a standby line, a lightning lane, and a single rider line.

### Standby line
* Open to anyone who has been admitted to the park.
* Generally the longest wait out of the three lines.
* Parties that enter the line together are usually able to ride the attraction seated together.

### Lighting lane
* Requires guests to make a reservation.
* Guests pay a fee for the ability to make reservations using the Disnetland mobile app.

### Single Rider Line
* Does not require a reservation.
* Each guest in the line is treated as a single party.
* Parties that enter the line together are usually not able ride the attraction seated together.
* The purpose of this line is to fill empty seats that cannot be filled by the stanby line or lighting lane due to the size of the parties not fitting in the remaining seats.
* Often a much shorter wait than the standby line.

## Motivation

Disneyland guests can access estimated wait times in real time for almost every attraction using the Dinsleyland mobile app. However, the app only displays the waiting time for the standby line and lightning lane but not the single rider line.

While waiting in the single rider line at Tiana's Bayou Adventure, I observed that the wait time is dependent on the liklihood that a space for a guest in the single rider line will become available. At this attaction, each log (vehicle that guests sit in) seats up to 6 people. The Disneyland cast members responsible for directing guests to seats primarily use the standby line and lightning lane to fill these seats. Guests in the single rider line are only able to board a log when there are open seat remaining. For example, if there is a party of 2 followed by a party of 3, there will be one seat remaining in the log. The cast member will direct a guest in the single rider line to fill this seat. On the contary, if there is a party of 2 followed by a party of 4, there will not be seats remaining and the guest next in line in the single rider queue will not be able to board.

Another factor that I oberved is that cast members do not immediately resort to the single rider line to fill open seats if the next party in the standby line or lightning lane does not fit into the remaining seats. Instead, they search several postions back in the line for a party that will fit.  For example, if there are three seats remaining, they search for a party of three in the standby line or lightning line by calling out "party of three" and waiting briefly for a party to respond and come forward. If there are no responses, only then are the remaining seats able to be boarded by guests in the single rider line.

Excited by a challenge, I wanted to model this dynamic and test whether I can create accurate estimates on the waiting times in the single rider queue at Tiana's Bayou Adventure that can be generalized for other Disneyland attractions.


## Assumptions

### Party Sizes
* Party sizes in the standby line can be any integer between 1 and 6.
* While it is possible that parties large than 6 will enter the line, I treat these as seperate parties that can broken up into parties smaller than 6.

### Party Size Probability
* For the wait time to be accurate, I need accurate measurements of the probabilties for each party size in the standby line.
* This data can be obtained from sampling party sizes that enter the standby line. However, I have not returned to the park since starting this project and have not yet attempted to collect this data. For now, I estimate the probability of various party sizes in the standby line.
* Observe below that a party of 1 is unlikely since any party of 1 can acheive a shorter wait time if they enter the single rider line without bearing the drawback of not riding with the rest of their party. Parties of 2 are very common since it is ideal for couples or friends. Parties of 3 and 4 are extremely frequent for families with one or two kids. Parties of 5 and 6 are less common but still occur.

### Call Out Policy

* I refer to policy of calling out for a certain size party to fill remaining seats as the "call out policy." The maxiumum number of parties that can hear the call out and feasibly reach the front of the line will represented by the variable called "policy." For this project, I set this number at 5. This can be easily adjusted as needed.

### Ride Capacity

* The ride capacity for Tiana's Bayou Adventure is 6. This can be adjusted to match other attractions.

In [ ]:
import numpy as np
import pandas as pd

party_sizes = [1, 2, 3, 4, 5, 6]
probabilties = [0.01, 0.24, 0.25, 0.3, 0.1, 0.1]
data = pd.DataFrame({ "Party Size": party_sizes, "Probability": probabilties })
data

policy = 5 #The maxiumum number of parties that can hear the call out and feasibly reach the front of the line
ride_capacity = 6 #The ride capacity for Tiana's Bayou Adventure is 6. This can be adjusted to match other attractions.

## Random Number Generation

The partysize() function generates a single random number between 1 and 6 based on the given probabilties above.

In [ ]:
def partysize():
    partysize = np.random.choice(data['Party Size'], p=data['Probability'])
    return partysize

## Fill Ride Function

The fill_ride() function:
1. Intializes the number of open seats in the ride as the maximum capacity the ride holds.
2. Creates a list of random numbers using the partysize() function where the lenth is the value given as the call-out policy.
3. The for loop looks at the next party in line and assigns them a seat. The loop then looks at the second party line and assigns them a seat if they fit. This continues up to the call-out policy amount. If there are no open seats remaining, the loop breaks instead of checking if the next parties can fit in the ride. This saves compute time since this function will be run a large number of times.
4. The function returns the amount of open seats remaining after the maximum number of parties allowed in the call out policy have been seated or attempted to be seated. The value returned is the number of open seats that will be filled by guests from the single rider queue.

In [ ]:
def fill_ride():
    open_seats = ride_capacity
    next_parties = [partysize() for i in range(policy)]

    for i in range(policy):
        if open_seats == 0:
            break
        if open_seats >= next_parties[i]:
            open_seats = open_seats - next_parties[i]

    return open_seats

To demonstrate how the above function works, I create an alternative version that takes a list as an argument instead of generating this list within the function. The function returns the number of open seats remaining as did the orginal function.

In [ ]:
def fill_ride_test(next_parties_list):
    open_seats = ride_capacity
    next_parties = next_parties_list

    for i in range(policy):
        if open_seats == 0:
            break
        if open_seats >= next_parties[i]:
            open_seats = open_seats - next_parties[i]

    return open_seats

Observe that in the following example, the first party takes up four seats. The rest of the parties are all larger than two and thus cannot be fit into the remaining two seats. Thus, the function returns the value 2 since there are two seats that will be filled by guests in the single rider line.

In [ ]:
fill_ride_test([4, 4, 3, 3, 3])

2

Observe that in the following example, there are zero open seats remaining since the first party takes up two seats and the remaining four seats are filled by the next party. The function returns zero since there will be zero guests from the single rider line used to fill the log.

In [ ]:
fill_ride_test([2, 4, 3, 5, 3])

0

Below are some other possible scenarios and the corresponding number of open seats remaining:

In [ ]:
fill_ride_test([2, 3, 3, 5, 2])

1

In [ ]:
fill_ride_test([4, 3, 3, 5, 2])

0

In [ ]:
fill_ride_test([3, 4, 6, 5, 4])

3

## Simulation

An instance of the fill_ride() function being called represents one attempt to fill a single log by using the next 5 parties in line and resorting to the single rider queue whenever there are open seats remaining. In some cases, there may be one or several open seats left to be filled by guests from the single rider queue. In other cases, there will be zero.

To determine the expected number of guests from the single rider queue that will be used on each log, I run the function one million times and compute the average.

Using this number and an estimate on the number of logs dispatched per minute, I can estimate the wait time for the single rider line for any given single rider queue length (number of guests in the single rider queue).

In [ ]:
num_trials = 1000000
total_single_riders = sum(fill_ride() for i in range(num_trials))

average = total_single_riders / num_trials
average

np.float64(0.608393)

## Convergence Test

I run the previous simulation 5 times to show that there is minimal variation (less than 0.01) in the result between simulations. I use the average result of these 5 simulations when computing the throughput (processing rate) of the single rider line.

In [ ]:
def simulation(num_tests, num_trials):
    k = num_tests
    n = num_trials

    tests = list(range(1, k + 1))
    averages = []

    for _ in tests:
        total_single_riders = sum(fill_ride() for i in range(n))
        average = total_single_riders / num_trials
        averages.append(average)

    df = pd.DataFrame({ "Test #": tests, "Average": averages })

    return df

result = simulation(5, 1000000)
result

,Test #,Average
0,1,0.609055
1,2,0.608622
2,3,0.607627
3,4,0.607877
4,5,0.608843


In [ ]:
x = sum(result['Average']) / len(result['Average'])
x

0.6084048000000001

In [ ]:
rides_per_min = 2

# Single Rider Line Throughput (measured in single riders per minute)
sr_throughput = x * rides_per_min
print("The single rider line processes an estimated", round(sr_throughput,4) , "guests per minute.")


The single rider line processes an estimated 1.2168 guests per minute.


## Final Result

Upon computing the throughput rate of the single rider line, I can determine the expected wait time by dividing the queue length by the throughput rate. Below is a table depicting the expected wait time for various queue lengths.

In [ ]:
intervals = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
wait_times = intervals / sr_throughput

df = pd.DataFrame({ "Queue Length": intervals, "Wait Time (Minutes)": wait_times })
df

,Queue Length,Wait Time (Minutes)
0,10,8.218213
1,20,16.436425
2,30,24.654638
3,40,32.872850
4,50,41.091063
5,60,49.309276
6,70,57.527488
7,80,65.745701
8,90,73.963913
9,100,82.182126
